In [78]:
import os
import pathlib
import zipfile
from openpyxl import load_workbook
from PIL import Image

def identify_sheet_content(sheet):
    has_images = any(sheet._images)
    has_linked_excel = any("excel" in rel.reltype for rel in sheet._rels)
    has_linked_pdf = any(cell.hyperlink.target.endswith('.pdf') for row in sheet.iter_rows() for cell in row if cell.hyperlink)

    if has_images:
        return 'image'
    elif has_linked_excel:
        return 'excel link'
    elif has_linked_pdf:
        return 'linked pdf'
    else:
        return 'tables'

def identify_excel_file(file_path):
    excel_file = load_workbook(file_path, data_only=True)
    sheet_types = {}

    for sheet_name in excel_file.sheetnames:
        sheet = excel_file[sheet_name]
        sheet_types[sheet_name] = identify_sheet_content(sheet)

    return sheet_types

# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file_path = 'C:/Users/Shreshtha/Downloads/Sample MNS Bartrack Report.xlsx'
result = identify_excel_file(excel_file_path)

for sheet_name, content_type in result.items():
    print(f"Sheet '{sheet_name}': {content_type}")

Sheet 'group 1': image
Sheet 'Sheet1': tables
Sheet 'group 2': tables
Sheet 'files': tables
Sheet 'Sheet6': tables
Sheet 'Sheet4': tables
Sheet 'Sheet2': tables
Sheet 'Sheet5': image
Sheet 'Sheet3': image


In [24]:
import mimetypes  

def extract_images_from_excel(path, output_folder_name='extracted_images_sample4'):
    """
    Extracts images from an Excel file and stores them in a single folder.

    Args:
        path (pathlib.Path or str): Excel file path.
        output_folder_name (str): Name of the folder to store the extracted images.
            Defaults to 'extracted_images_again'.

    Returns:
        new_paths (list[pathlib.Path]): List of paths to extracted images.
    """
    # Convert path to pathlib.Path if it's a string
    if isinstance(path, str):
        path = pathlib.Path(path)

    # Check if the file has the '.xlsx' extension
    if path.suffix != '.xlsx':
        raise ValueError('Path must be an xlsx file')

    # Extract the filename (excluding the extension) using .stem
    name = path.stem

    # Create a new folder for the extracted images
    output_folder = path.parent / output_folder_name
    output_folder.mkdir(exist_ok=True)  

    # Create a temporary directory for unzipping the Excel file
    temp_dir = path.parent / 'temp'
    temp_dir.mkdir(exist_ok=True) 

    try:
        # Unzip the Excel file into the temporary directory
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Locate the 'media' directory within the unzipped content
        media_dir = temp_dir / 'xl' / 'media'

        image_index = 0  # Initialize an index for the images
        new_paths = []  # List to store the paths of the extracted images

        # Iterate through the files in the 'media' directory
        for root, dirs, files in os.walk(media_dir):
            for file in files:
                # Determine the MIME type of the file
                mime_type, encoding = mimetypes.guess_type(file)

                # Check if the file is an image based on MIME type and file extension
                if mime_type and mime_type.startswith('image') and file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_index += 1  # Increment the image index for each image found

                    # Construct paths for the original image and the new destination
                    image_path = pathlib.Path(root) / file
                    new_path = output_folder / f'{name}-{str(image_index)}.png'

                    # Copy the image to the output folder with a new name
                    shutil.copy(image_path, new_path)

                    # Store the new path in the list
                    new_paths.append(new_path)

    finally:
        # Cleanup: Remove the temporary directory
        shutil.rmtree(temp_dir)

    # Return the list of paths to the extracted images
    return new_paths


In [25]:
excel_file_path = "C:/Users/Shreshtha/Downloads/Sample MNS Bartrack Report.xlsx"
extracted_image_paths = extract_images_from_excel(excel_file_path)

In [26]:
# Path to the directory containing the images
images_directory = "C:/Users/Shreshtha/Downloads/extracted_images_sample4"

In [28]:
import cv2
import os
import shutil
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def resize_image(image, target_size=(300, 300)):
    try:
        if image is not None and image.size != 0:
            return cv2.resize(image, target_size)
        else:
            return image
    except Exception as e:
        print(f"Error resizing image: {e}")
        return None

def extract_features(image_path, target_size=(300, 300)):
    try:
        image = cv2.imread(image_path)
        resized_image = resize_image(image, target_size)

        if resized_image is not None:
            # Convert the image to grayscale
            gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

            # Flatten the 2D array into a 1D array
            flattened = gray.flatten()

            return flattened
        else:
            return None
    except Exception as e:
        print(f"Error extracting features: {e}")
        return None

def find_optimal_clusters(input_folder, max_clusters=10):
    try:
        image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

        feature_vectors = []

        for img_file in image_files:
            features = extract_features(os.path.join(input_folder, img_file))
            if features is not None:
                feature_vectors.append(features)

        if feature_vectors:
            feature_vectors = np.array(feature_vectors)

            # Apply k-means clustering for different values of k
            inertias = []
            for k in range(1, max_clusters + 1):
                kmeans = KMeans(n_clusters=k, random_state=42)
                kmeans.fit(feature_vectors)
                inertias.append(kmeans.inertia_)

            # Plot the elbow method curve
            plt.plot(range(1, max_clusters + 1), inertias, marker='o')
            plt.xlabel('Number of Clusters')
            plt.ylabel('Inertia (within-cluster sum of squares)')
            plt.title('Elbow Method for Optimal Number of Clusters')
            plt.show()

    except Exception as e:
        print(f"Error finding optimal clusters: {e}")

# Specify your input folder path
input_folder_path = "C:/Users/Shreshtha/Downloads/extracted_images_sample4"

# Call the function to find the optimal number of clusters
find_optimal_clusters(input_folder_path, max_clusters=10)


C:\Users\Shreshtha\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Shreshtha\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Shreshtha\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Error finding optimal clusters: n_samples=3 should be >= n_clusters=4.


In [31]:
import os 
from google.api_core.client_options import ClientOptions
from google.cloud import documentai 
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:/Users/Shreshtha/Downloads/balmy-outcome-412805-7a02be612f44.json"

project_id = 'balmy-outcome-412805' 
location = 'us' 
processor_id = '1c327dd87f42b98b' 
processor_version = 'rc' 
local_file_path = "C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-3.png"  
mime_type = 'image/png'

def online_process(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    """
    A function to process a document online using Google Document AI.
    """

    # Define an options dictionary, which includes the API's URL. This is used to connect to Google's Document AI service
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}

    # Create a Document AI client, think of it as our bridge for communicating with Google's services
    documentai_client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # Generate the complete name of the processor
    # You need to first create a processor in the Google Cloud console
    resource_name = documentai_client.processor_path(project_id, location, processor_id)

    # Read in the document you want to analyze (like an image or PDF), and store it in the variable image_content
    with open(file_path, "rb") as image:
        image_content = image.read()

        # Convert the read document into a format that Google Document AI can understand, i.e., a RawDocument object
        raw_document = documentai.RawDocument(
            content=image_content, mime_type=mime_type
        )
        # Create a request, which includes the name of the processor and the document we want to analyze
        request = documentai.ProcessRequest(
            name=resource_name, raw_document=raw_document
        )
        # Send our request and receive the analysis results
        result = documentai_client.process_document(request=request)

        # Return this analysis result
        return result.document

def trim_text(text: str): 
    """ Removes spaces and newline characters. """ 
    return text.strip().replace("\n", " ")

# Use the local file path in your function
document = online_process(
    project_id=project_id,
    location=location,
    processor_id=processor_id,
    file_path=local_file_path,  # Use the local path here
    mime_type=mime_type,
)

names = []
name_confidence = []
values = []
value_confidence = []

for page in document.pages:
    for field in page.form_fields:
        names.append(trim_text(field.field_name.text_anchor.content))
        name_confidence.append(field.field_name.confidence)
        values.append(trim_text(field.field_value.text_anchor.content))
        value_confidence.append(field.field_value.confidence)

df = pd.DataFrame(
    {
        "Field Name": names,
        "Field Value": values
    }
)


In [32]:
df

,Field Name,Field Value
0,Specialist:,No Charge after Deductible
1,Family M.O.O.P.,"$8,000"
2,PCP:,No Charge after Deductible
3,Family M.O.O.P.,$11.700
4,Inpatient:,No Charge after Deductible
5,Coinsurance:,None
6,Outpatient Freestanding:,No Charge after Deductible
7,Family Deductible:,"$5,700"
8,Financial Accumulation Period:,Calendar Year
9,ER Copay:,No Charge after Deductible


In [38]:
import os 
from google.api_core.client_options import ClientOptions
from google.cloud import documentai 
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:/Users/Shreshtha/Downloads/balmy-outcome-412805-7a02be612f44.json"

project_id = 'balmy-outcome-412805' 
location = 'us' 
processor_id = '1c327dd87f42b98b' 
processor_version = 'rc' 
local_file_path = "C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-2.png"  
mime_type = 'image/png'

def online_process(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    """
    A function to process a document online using Google Document AI.
    """

    # Define an options dictionary, which includes the API's URL. This is used to connect to Google's Document AI service
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}

    # Create a Document AI client, think of it as our bridge for communicating with Google's services
    documentai_client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # Generate the complete name of the processor
    # You need to first create a processor in the Google Cloud console
    resource_name = documentai_client.processor_path(project_id, location, processor_id)

    # Read in the document you want to analyze (like an image or PDF), and store it in the variable image_content
    with open(file_path, "rb") as image:
        image_content = image.read()

        # Convert the read document into a format that Google Document AI can understand, i.e., a RawDocument object
        raw_document = documentai.RawDocument(
            content=image_content, mime_type=mime_type
        )
        # Create a request, which includes the name of the processor and the document we want to analyze
        request = documentai.ProcessRequest(
            name=resource_name, raw_document=raw_document
        )
        # Send our request and receive the analysis results
        result = documentai_client.process_document(request=request)

        # Return this analysis result
        return result.document

def trim_text(text: str): 
    """ Removes spaces and newline characters. """ 
    return text.strip().replace("\n", " ")

# Use the local file path in your function
document = online_process(
    project_id=project_id,
    location=location,
    processor_id=processor_id,
    file_path=local_file_path,  # Use the local path here
    mime_type=mime_type,
)

names = []
name_confidence = []
values = []
value_confidence = []

# Iterate through pages and form fields
for page in document.pages:
    for form_field in page.form_fields:
        # Check if the form field represents a table cell
        if form_field.field_name.text_anchor.text_segments[0].start_index == 0 and form_field.field_name.text_anchor.text_segments[0].end_index == 0:
            # Extract table cell information
            table_cell = form_field.field_value.table_cell
            row_index = table_cell.row_index
            col_index = table_cell.col_index
            content = trim_text(table_cell.content)

            # Use row and column indices to represent the cell location
            cell_name = f"Row_{row_index}_Col_{col_index}"
            names.append(cell_name)
            name_confidence.append(1.0)  # You can adjust confidence as needed
            values.append(content)
            value_confidence.append(form_field.field_value.confidence)

# Create a DataFrame from the extracted table data
df = pd.DataFrame(
    {
        "Cell Name": names,
        "Cell Value": values
    }
)


In [39]:
df

,Cell Name,Cell Value


In [46]:

from typing import Optional, Sequence

from google.api_core.client_options import ClientOptions
from google.cloud import documentai

# TODO(developer): Uncomment these variables before running the sample.
project_id = 'balmy-outcome-412805' 
location = 'us' 
processor_id = '1c327dd87f42b98b' 
processor_version = 'rc' 
local_file_path = "C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-1.png"  
mime_type = 'image/png'

def process_document_form_sample(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    # Online processing request to Document AI
    document = process_document(
        project_id, location, processor_id, processor_version, file_path, mime_type
    )

    # Read the table and form fields output from the processor
    # The form processor also contains OCR data. For more information
    # on how to parse OCR data please see the OCR sample.

    text = document.text
    print(f"Full document text: {repr(text)}\n")
    print(f"There are {len(document.pages)} page(s) in this document.")


In [79]:
# Replace these values with your actual configuration
project_id = 'balmy-outcome-412805' 
location = 'us' 
processor_id = '1c327dd87f42b98b' 
processor_version = 'rc' 
local_file_path = "C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-1.png"  
mime_type = 'image/png'

# Call the function to process the document and extract information
processed_document = process_document_form_sample(
    project_id, location, processor_id, processor_version, local_file_path, mime_type
)

# Example: Print information about tables
for page in processed_document.pages:
    print(f"\n\n**** Page {page.page_number} ****")
    print(f"Found {len(page.tables)} table(s):")

    for table in page.tables:
        num_columns = len(table.header_rows[0].cells)
        num_rows = len(table.body_rows)
        print(f"Table with {num_columns} columns and {num_rows} rows:")

        # Print header rows
        print("Columns:")
        for cell in table.header_rows[0].cells:
            print(f"{repr(cell.layout.text_anchor.content.strip())} | ", end="")
        print("\nTable body data:")
        for table_row in table.body_rows:
            row_text = ""
            for cell in table_row.cells:
                row_text += f"{repr(cell.layout.text_anchor.content.strip())} | "
            print(row_text)


Full document text: 'ICICI Lombard\nMax Bupa\nApollo Munich\nTata AIG\nIhealth\nHeartbeat Gold\nOptima Restore\nMediPrime\nStar Health Comprehensive\nBasic sum insured\n10 lacs\n10 lacs\n10 lacs\n10 lacs\n10 lacs\nPremium\n10,643\n22,696\n13,607\n12,205\n17,483\nHosptalization benefits\nPre-existing diseases:\nPre-existing diseases:\nPre-existing diseases: 3\nPre-existing diseases:\n2 years\n2 years\nyears\nPre-existing diseases:\n4 years\nSpecific illnesses/\n4 years\nSpecific illnesses/\nSpecific\nSpecific illnesses/\nSpecific illnesses/\n1 Waiting period\ntreatments: 2 years\nillnesses/treatments: None\ntreatments: 2 years\ntreatments: 2 years\ntreatments: 2 years\nNo restriction\nNo restriction\nNo restriction\nNo restriction\nNo restriction\n2 Hospital accomodation\n/sub-limits\n/sub-limits\n/sub-limits\n/sub-limits\n/sub-limits\n60 days if informed of\n3 Pre-hospitalization\n30 days\n30 days\n60 days\n30 days\nhospitalization 5 days in\nadvance, else 30 days\n90 days if informed 

In [69]:
# Replace these values with your actual configuration
project_id = 'balmy-outcome-412805' 
location = 'us' 
processor_id = '1c327dd87f42b98b' 
processor_version = 'rc' 
local_file_path = "C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-2.png"  
mime_type = 'image/png'

# Call the function to process the document and extract information
processed_document = process_document_form_sample(
    project_id, location, processor_id, processor_version, local_file_path, mime_type
)

# Example: Print information about tables
for page in processed_document.pages:
    print(f"\n\n**** Page {page.page_number} ****")
    print(f"Found {len(page.tables)} table(s):")

    for table in page.tables:
        num_columns = len(table.header_rows[0].cells)
        num_rows = len(table.body_rows)
        print(f"Table with {num_columns} columns and {num_rows} rows:")

        # Print header rows
        print("Columns:")
        for cell in table.header_rows[0].cells:
            print(f"{repr(cell.layout.text_anchor.content.strip())} | ", end="")
        print("\nTable body data:")
        for table_row in table.body_rows:
            row_text = ""
            for cell in table_row.cells:
                row_text += f"{repr(cell.layout.text_anchor.content.strip())} | "
            print(row_text)


Full document text: 'ABC Ltd\nGroup Name\nGroup Number\n123\nRenewal Date\n01-01-2024\n20\nNo of employees enrolled\nGroup State\nCT\nMarket\nLarge\nIn Network (IN):\nCopay\n30/45\nIN Coinsurance %\n100/0\nIN Deductible\n2500/5000\nOut of Network (OON):\nOON Coinsurance %\n100/0\nOON Deductible\n2500/5000\n'

There are 1 page(s) in this document.


**** Page 1 ****

Found 2 table(s):
Table with 1 columns and 8 rows:
Columns:
'20\nNo of employees enrolled\nGroup State\nCT' | 
Table body data:
'Market\nLarge' | 
'In Network (IN):' | 
'Copay\n30/45' | 
'IN Coinsurance %\n100/0' | 
'IN Deductible\n2500/5000' | 
'Out of Network (OON):' | 
'OON Coinsurance %\n100/0' | 
'OON Deductible\n2500/5000' | 
Table with 2 columns and 2 rows:
Columns:
'Copay' | '30/45' | 
Table body data:
'IN Coinsurance %' | '100/0' | 
'IN Deductible' | '2500/5000' | 

Found 7 form field(s):
    * '30/45': 'Copay'
    * '100/0': 'IN Coinsurance %'
    * '100/0': '2500/5000'
    * 'CT': 'Group State'
    * 'Market': 'L

In [74]:
json_key_file_path = "C:/Users/Shreshtha/Downloads/balmy-outcome-412805-e9aa761e058c.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_file_path

In [76]:
from google.cloud import vision_v1
from google.cloud.vision_v1 import types

def detect_text(image_path):
    client = vision_v1.ImageAnnotatorClient()

    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)
    response = client.text_detection(image=image)

    texts = response.text_annotations
    for text in texts:
        print(f'"{text.description}"\n')

# Replace 'path/to/your/image.jpg' with the actual image path
detect_text("C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-1.png")


"Basic sum insured
Premium
Hosptalization benefits
1 Waiting period
2 Hospital accomodation
3 Pre-hospitalization
4 Post hospitalization
5 Hospital cash/Daily Cash
6 Emergency ambulance
7 Organ donor expense
Co-payment feature
8/Annual deductible
9 Day care procedures
10 Domicilliary hospitalization
11 Alternative treatments
Maternity Benefits
Maternity benefits
12 (delivery expenses)
13 New born baby cover
Renewal benefits
14 Renewal benefits
15 Loading on claims
16 Health checkup
ICICI Lombard
Ihealth
10 lacs
10,643
Pre-existing diseases:
2 years
Specific illnesses/
treatments: 2 years
No restriction
/sub-limits
30 days
60 days after
No
Rs. 1500 per
hospitalization
Not covered
Not applicable
140 day care procedures
covered
No
No
Waiting period of 36 months
None
No Claim:
Additional 10% sum insured
at the time of renewal for
every claim free year.
In case of a claim,
cumulative additional sum
insured to go down by 50%
ΝΑ
Yes
Max Bupa
Heartbeat Gold
10 lacs
22,696
Pre-existing diseases

In [81]:
from PIL import Image
import pytesseract

def extract_text_from_image(image_path):
    try:
        # Open the image file
        with Image.open(image_path) as img:
            # Use pytesseract to do OCR on the image
            text = pytesseract.image_to_string(img)

            # Print the extracted text
            print(text)
    except Exception as e:
        print(f"Error: {e}")

# Replace 'path/to/your/image.jpg' with the actual image path
extract_text_from_image("C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-1.png")


lWaiting period

Pre-existing disease:
2years
Specific illnesses/
treatments: 2 years

Pre-existing disease:
2years
Specific

illnesses/treatment

None

Pre-existing diseases:
years

Specificilinesses/

treatments: 2 years

ICICI Lombard Max Bupa Apollo Munich Tata AIG

thealth Heartbeat Gold Optima Restore MediPrime Star Health Comprehensive
[Basic sum insured 10 lacs 10 lacs 10 lacs 10 lacs 10 lacs
Premium 22,696 13,607 12,205 17,483

Pre-existing disease:
years

Specific illnesses/

treatments: 2 years

Pre-existing disease:
years
Specificillnesses/
treatments: 2 years

No restriction No restriction No restriction No restriction No restriction
2|Hospital accomodation Jsub-limits
60 days if informed of
hospitalization 5 days in
3|Pre-hospitalization 30 days 30 days 60 days advance, else 30 days 30 days
90 days if informed of
hospitalization 5 days in
4|Post hospitalization 60 days after 60 days after 180 days advance, else 60 days 60 days
Only in case of shared Only for accompanying


In [90]:
from PIL import Image
import pytesseract
import pandas as pd
from IPython.display import display

def extract_and_convert_to_dataframe(image_path):
    try:
        # Open the image file
        with Image.open(image_path) as img:
            # Use pytesseract to do OCR on the image
            text = pytesseract.image_to_string(img)

            # Split the lines of the extracted text
            lines = text.split('\n')

            # Process lines to create a list of lists for DataFrame
            data = [line.split() for line in lines if line.strip()]

            # Create a DataFrame from the processed data
            df = pd.DataFrame(data)

            # Display the DataFrame in a more organized way
            print("DataFrame from Extracted Text:")
            display(df)
            
    except Exception as e:
        print(f"Error: {e}")

# Replace 'path/to/your/image.jpg' with the actual image path
extract_and_convert_to_dataframe("C:/Users/Shreshtha/Downloads/extracted_images_sample4/Sample MNS Bartrack Report-1.png")


DataFrame from Extracted Text:


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,lWaiting,period,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Pre-existing,disease:,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2years,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Specific,illnesses/,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,treatments:,2,years,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,(Once,every,four,years,Once,every,three,None,None,None,...,None,None,None,None,None,None,None,None,None,None
88,16|Health,checkup,Yes,Yes,(Annual),No,claim,free,years,claim,...,None,None,None,None,None,None,None,None,None,None
89,17|Claim,settlement,record,96.9%,82.2%,79.0%,84.8%,69.0%,None,None,...,None,None,None,None,None,None,None,None,None,None
90,‘Source:,Policy,documents,for,the,respective,policies,(December,2014).,For,...,calculated,as,100%-(%élaims,repudiated,+5,claims,pending,for,over,six
